
From the book (chapter 14.5):

>The US Federal Election Commission publishes data on contributions to political campaigns. This includes contributor names, occupation and employer, address, and contribution amount. An interesting dataset is from the 2012 US presidential election.

___



In [ ]:
library(tidyverse)
library(magrittr)


___

## Loading the data

This is a version of the above-mentioned dataset downloaded in June 2012.



In [ ]:
fec <- read_csv("https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/fec/P00000001-ALL.csv")

In [ ]:
fec


We can add another column to report if the candidate is a Republican or Democrat. In order to do that, we will simply add `"Democrat"` to every Barack Obama entry, and `"Republican"` to all the others.



In [ ]:
fec %<>%
    mutate(party = case_when(cand_nm == "Obama, Barack" ~ "Democrat",
                             TRUE ~ "Republican"))


Let's restrict the dataset to only positive contributions, removing refunds.



In [ ]:
fec %<>%
    filter(contb_receipt_amt > 0)

In [ ]:
fec



## Donation statistics by Occupation and Employer

Let's see if there is any trend of donations amount based on donators occupation and employer.



In [ ]:
fec %>%
    group_by(contbr_occupation) %>%
    summarise(count = n()) %>%
    arrange(desc(count))

In [ ]:
fec %>%
    group_by(contbr_employer) %>%
    summarise(count = n()) %>%
    arrange(desc(count))


We should clean a bit these data, because some of them refer to the same type of employment but are encoded differently.



In [ ]:
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "INFORMATION REQUESTED PER BEST EFFORTS",
                   replacement = "NOT PROVIDED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "INFORMATION REQUESTED",
                   replacement = "NOT PROVIDED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "INFORMATION REQUESTED (BEST EFFORTS)",
                   replacement = "NOT PROVIDED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^NONE$",
                   replacement = "NOT PROVIDED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^REFUSED$",
                   replacement = "NOT PROVIDED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^REQUESTED$",
                   replacement = "NOT PROVIDED")
fec %<>% replace_na(list(contbr_employer = "NOT PROVIDED",
                         contbr_occupation = "NOT PROVIDED"))
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^N/A$",
                   replacement = "NOT PROVIDED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^C.E.O.$",
                   replacement = "CEO")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^RN$",
                   replacement = "REGISTERED NURSE")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^R.N.$",
                   replacement = "REGISTERED NURSE")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^UNEMPLOYED$",
                   replacement = "NOT EMPLOYED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "^SELF$",
                   replacement = "SELF-EMPLOYED")
fec %<>% mutate_if(is_character, str_replace_all,
                   pattern = "SELF EMPLOYED",
                   replacement = "SELF-EMPLOYED")

In [ ]:
fec %>%
    group_by(contbr_occupation) %>%
    summarise(count = n()) %>%
    arrange(desc(count))

In [ ]:
fec %>%
    group_by(contbr_employer) %>%
    summarise(count = n()) %>%
    arrange(desc(count))


After this cleaning, let's create another dataset where we remove all candidates but Barack Obama and Mitt Romney, the main two candidates for this election.



In [ ]:
fec_mrbo <- fec %>%
    filter(cand_nm %in% c("Obama, Barack", "Romney, Mitt"))

In [ ]:
fec_mrbo



Now we can aggregate the data by occupation and party, and further filter it to keep only occupations that donated over $2 million.



In [ ]:
by_occupation <- fec %>%
    group_by(contbr_occupation, party) %>%
    summarise(donations = sum(contb_receipt_amt))

In [ ]:
over_2mm <- by_occupation %>%
    filter(donations >= 2000000)


Let's view these data.



In [ ]:
over_2mm %>%
    ggplot(aes(x = reorder(contbr_occupation, donations))) +
    geom_col(aes(y = donations, fill = party), position = "dodge") +
    coord_flip() +
    labs(x = "Occupation", y = "Donations (US$)", fill = "Party",
         title = "Total donations by party per occupation") +
    theme(legend.position = c(0.9, 0.14),
          legend.background = element_rect(fill = "transparent"))


Now we might be interested in aggregating these data by candidate name and occupation or employer, to find the top 10 occupations and employers who donated to the two candidates.



In [ ]:
occupation_grouped <- fec_mrbo %>%
    group_by(cand_nm, contbr_occupation) %>%
    summarise(donations = sum(contb_receipt_amt))
top_10_occup <- top_n(occupation_grouped, 10, donations)

In [ ]:
employer_grouped <- fec_mrbo %>%
    group_by(cand_nm, contbr_employer) %>%
    summarise(donations = sum(contb_receipt_amt))
top_10_empl <- top_n(employer_grouped, 10, donations)


Let's now visualize these two sets.



In [ ]:
top_10_occup %>%
    ggplot(aes(x = reorder(contbr_occupation, donations))) +
    geom_col(aes(y = donations, fill = cand_nm), position = "dodge") +
    coord_flip() +
    labs(x = "Occupation", y = "Donations (US$)", fill = "Candidate",
         title = "Top 10 donating occupations") +
    theme(legend.position = c(0.88, 0.14),
          legend.background = element_rect(fill = "transparent"))

In [ ]:
top_10_empl %>%
    ggplot(aes(x = reorder(contbr_employer, donations))) +
    geom_col(aes(y = donations, fill = cand_nm), position = "dodge") +
    coord_flip() +
    labs(x = "Employer", y = "Donations (US$)", fill = "Candidate",
         title = "Top 10 donating employers") +
    theme(legend.position = c(0.86, 0.14),
          legend.background = element_rect(fill = "transparent"))



## Bucketing donation amounts

We can discretize the contribution amounts into bins describing the contribution size.



In [ ]:
fec_mrbo %<>%
    mutate(contb_bin = cut(contb_receipt_amt,
                           breaks = c(0, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000)))


Now it is possible to aggregate these data by candidate and contribution bin, to get a histogram by donation size.



In [ ]:
contrib_grouped <- fec_mrbo %>%
    group_by(contb_bin, cand_nm) %>%
    summarise(donations = n()) %>%
    mutate(donation_perc = donations / sum(donations))

In [ ]:
contrib_grouped

In [ ]:
contrib_grouped %>%
    filter(donation_perc != 1.0) %>%
    ggplot(aes(x = contb_bin)) +
    geom_col(aes(y = donation_perc, fill = cand_nm), position = "dodge") +
    coord_flip() +
    labs(x = "Donation bins", y = "Percentage of donations", fill = "Candidate",
         title = "Percentage of donations received per each donation size") +
    scale_x_discrete(labels = c("(0, 1]", "(1, 10]", "(10, 100]", "(100, 1000]", "(1000, 10000]", "(10000, 100000]")) +
    scale_y_continuous(breaks = c(0.0, 0.2, 0.4, 0.6, 0.8)) +
    theme(legend.position = c(0.88, 0.88),
          legend.background = element_rect(fill = "transparent"))



## Donation statistics by State

Let's aggregate the data by candidate name and state.



In [ ]:
state_grouped <- fec_mrbo %>%
    group_by(contbr_st, cand_nm) %>%
    summarise(donations = sum(contb_receipt_amt)) %>%
    mutate(donation_perc = donations / sum(donations))

In [ ]:
state_grouped


And finally a (quite messy) visualization of these data.



In [ ]:
state_grouped %>%
    filter(donation_perc != 1.0, donation_perc >= 0.1, donation_perc <= 0.9) %>%
    ggplot(aes(x = contbr_st)) +
    geom_col(aes(y = donation_perc, fill = cand_nm), position = "dodge") +
    coord_flip() +
    labs(x = "US State", y = "Percentage of donations", fill = "Candidate",
         title = "Percentage of donations by state") +
    theme(legend.position = c(0.92, 0.56),
          legend.background = element_rect(fill = "transparent"))


___



In [ ]:
sessionInfo()